## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import matplotlib.pyplot as plt

# Import API key
from config import g_key
from config import weather_api_key
from citipy import citipy

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_data/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lengshuitan,CN,26.4111,111.5956,73.35,97,100,2.26,overcast clouds
1,1,Richards Bay,ZA,-28.7830,32.0377,68.58,78,0,10.76,clear sky
2,2,Hermanus,ZA,-34.4187,19.2345,60.80,79,1,19.60,clear sky
3,3,Hilo,US,19.7297,-155.0900,83.28,75,100,6.91,overcast clouds
4,4,Mayo,US,38.8876,-76.5119,80.35,55,0,9.22,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
print("What is the maximum temperature you want?")
maxTemp = float(input("max temp: "))

print("What is the minimum temperature you want?")
minTemp = float(input("minimum temp: "))

What is the maximum temperature you want?
max temp: 90
What is the minimum temperature you want?
minimum temp: 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_df = city_data_df.loc[(city_data_df["Max Temp"] > minTemp) & (city_data_df["Max Temp"] < maxTemp)]

cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Hilo,US,19.7297,-155.0900,83.28,75,100,6.91,overcast clouds
4,4,Mayo,US,38.8876,-76.5119,80.35,55,0,9.22,clear sky
5,5,Port Hedland,AU,-20.3167,118.5667,75.25,94,100,8.05,light rain
7,7,Dhanwar,IN,24.4167,85.9833,84.90,44,0,5.06,clear sky
10,10,Hambantota,LK,6.1241,81.1185,78.89,83,71,13.69,broken clouds


In [5]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Hilo,US,83.28,overcast clouds,19.7297,-155.0900,
4,Mayo,US,80.35,clear sky,38.8876,-76.5119,
5,Port Hedland,AU,75.25,light rain,-20.3167,118.5667,
7,Dhanwar,IN,84.90,clear sky,24.4167,85.9833,
10,Hambantota,LK,78.89,broken clouds,6.1241,81.1185,
12,Kupang,ID,75.20,scattered clouds,-10.1667,123.5833,
20,Takoradi,GH,79.93,broken clouds,4.8845,-1.7554,
24,Georgetown,MY,81.95,few clouds,5.4112,100.3354,
25,Puerto Ayora,EC,80.56,broken clouds,-0.7393,-90.3518,
29,Andalan,PH,79.05,overcast clouds,5.9594,121.3903,


In [6]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_URL, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found - skipping.")
        

Hotel not found - skipping.
Hotel not found - skipping.
Hotel not found - skipping.
Hotel not found - skipping.
Hotel not found - skipping.
Hotel not found - skipping.
Hotel not found - skipping.
Hotel not found - skipping.
Hotel not found - skipping.
Hotel not found - skipping.
Hotel not found - skipping.
Hotel not found - skipping.
Hotel not found - skipping.


In [7]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=["Hotel Name"], inplace=True)
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Hilo,US,83.28,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
4,Mayo,US,80.35,clear sky,38.8876,-76.5119,Resorts Resource Group International
5,Port Hedland,AU,75.25,light rain,-20.3167,118.5667,The Esplanade Hotel
7,Dhanwar,IN,84.90,clear sky,24.4167,85.9833,K G N HOTEL
10,Hambantota,LK,78.89,broken clouds,6.1241,81.1185,Bungalow 63
12,Kupang,ID,75.20,scattered clouds,-10.1667,123.5833,Hotel On The Rock by Prasanthi
20,Takoradi,GH,79.93,broken clouds,4.8845,-1.7554,Raybow International Hotel
24,Georgetown,MY,81.95,few clouds,5.4112,100.3354,Cititel Penang
25,Puerto Ayora,EC,80.56,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
33,Garden City,US,75.88,broken clouds,42.3256,-83.3310,Big Trees Mobile Home Community


In [8]:
# 8a. Create the output File (CSV)
output_data_file = "weather_data/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig
plt.savefig("weather_data\WeatherPy_vacation_map.png")

<Figure size 432x288 with 0 Axes>